# Using Convolutional Neural Networks

Welcome to the first week of the first deep learning certificate! We're going to use convolutional neural networks (CNNs) to allow our computer to see - something that is only possible thanks to deep learning.

## Introduction to this week's task: 'Dogs vs Cats'

We're going to try to create a model to enter the [Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats) competition at Kaggle. There are 25,000 labelled dog and cat photos available for training, and 12,500 in the test set that we have to try to label for this competition. According to the Kaggle web-site, when this competition was launched (end of 2013): *"**State of the art**: The current literature suggests machine classifiers can score above 80% accuracy on this task"*. So if we can beat 80%, then we will be at the cutting edge as at 2013!

## Basic setup

There isn't too much to do to get started - just a few simple configuration steps.

This shows plots in the web page itself - we always wants to use this when using jupyter notebook:

In [1]:
%matplotlib inline

In [6]:
ls ../../data/lesson_1/redux_dogscats/

sample_submission.csv  test/  test.zip  train/  train.zip  valid/


Define path to data: (It's a good idea to put it in a subdirectory of your notebooks folder, and then exclude that directory from git control by adding it to .gitignore.)

In [7]:
path = "../../data/lesson_1/redux_dogscats/"
#path = "data/dogscats/sample/"

A few basic libraries that we'll need for the initial exercises:

In [8]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

We have created a file most imaginatively called 'utils.py' to store any little convenience functions we'll want to use. We will discuss these as we use them.

In [9]:
import utils; reload(utils)
from utils import plots

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [10]:
import keras

# Use a pretrained VGG model with our **Vgg16** class

Our first step is simply to use a model that has been fully created for us, which can recognise a wide variety (1,000 categories) of images. We will use 'VGG', which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (*VGG  19*) and a smaller, faster model (*VGG 16*). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

We have created a python class, *Vgg16*, which makes using the VGG 16 model very straightforward. 

## The punchline: state of the art custom model in 7 lines of code

Here's everything you need to do to get >97% accuracy on the Dogs vs Cats dataset - we won't analyze how it works behind the scenes yet, since at this stage we're just going to focus on the minimum necessary to actually do useful work.

In [11]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [12]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [13]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Epoch 1/1
22500/22500 [==============================] - 608s - loss: 0.1202 - acc: 0.9680 - val_loss: 0.0463 - val_acc: 0.9860

In [153]:
vgg.fit(batches, val_batches, nb_epoch=8)

Epoch 1/8
22500/22500 [==============================] - 591s - loss: 0.1076 - acc: 0.9802 - val_loss: 0.0585 - val_acc: 0.9892

In [158]:
test_batches = vgg.get_batches(path + 'test', shuffle=False, batch_size=64, class_mode=None)
file_names = test_batches.filenames

Found 12500 images belonging to 1 classes.


In [159]:
predictions = vgg.model.predict_generator(test_batches, val_samples=12500)

In [161]:
predictions[:, 1]

array([  4.4402e-20,   6.9843e-05,   7.7544e-14, ...,   6.8056e-22,   3.0750e-24,   1.0000e+00], dtype=float32)

In [162]:
import pandas as pd
df = pd.DataFrame(zip(file_names, predictions[:,1]), columns=['id', 'label'])
df['id'] = df['id'].apply(lambda i: i.split('/')[1].split('.')[0])

In [163]:
df.to_csv('submission_probabilities.csv', index=False)